In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading and Preprocessing
df = pd.read_csv("logistics_delay_dataset.csv")
# Features and target (focusing on actionable feature truck_allocation)
features = [
    "weather_severity", "traffic_congestion", "warehouse_congestion",
    "driver_experience", "truck_allocation", "route_complexity",
    "package_volume", "time_sensitivity"
]
X = df[features]
y = df["delayed"]

# Split into training set and intervention experiment set (using subset marked with intervention)
intervention_mask = df["split"] == "intervention"  # Assuming dataset has split column marking intervention subset
X_interv, y_interv = X[intervention_mask], y[intervention_mask]
X_train, X_val, y_train, y_val = train_test_split(X[~intervention_mask], y[~intervention_mask], test_size=0.2, random_state=42)

# 2. Train Baseline Model
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
print("Baseline Model AUC on Validation Set:", round(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]), 3))

# 3. Construct Virtual Intervention Experiment
def causal_consistency_test(model, X_interv, actionable_feature="truck_allocation"):
    """
    Causal consistency test: Change actionable feature and check if prediction direction aligns with business logic
    Business logic: When truck_allocation increases from 0→1 (more trucks), delay probability should decrease
    """
    # Copy experiment data, set actionable feature to 0 and 1 respectively
    X_interv_0 = X_interv.copy()
    X_interv_0[actionable_feature] = 0  # Before intervention: no additional trucks
    X_interv_1 = X_interv.copy()
    X_interv_1[actionable_feature] = 1  # After intervention: add trucks
    
    # Predict delay probabilities
    prob_0 = model.predict_proba(X_interv_0)[:, 1]
    prob_1 = model.predict_proba(X_interv_1)[:, 1]
    
    # Calculate proportion of samples consistent with business direction (prob_1 < prob_0)
    consistent_ratio = np.mean(prob_1 < prob_0)
    # Calculate average probability change
    avg_prob_change = np.mean(prob_0 - prob_1)
    
    return {
        "Proportion consistent with business direction": round(consistent_ratio, 3),
        "Average reduction in delay probability": round(avg_prob_change, 3),
        "prob_0 (before intervention)": prob_0,
        "prob_1 (after intervention)": prob_1
    }

# Execute causal consistency test
test_result = causal_consistency_test(model, X_interv)
print("\nCausal Consistency Test Results:")
for k, v in test_result.items():
    if isinstance(v, float):
        print(f"{k}: {v}")

# Determine consistency (threshold: proportion ≥ 0.7 considered consistent)
is_consistent = test_result["Proportion consistent with business direction"] >= 0.7
print(f"\nCausal Consistency Conclusion: {'Consistent' if is_consistent else 'Inconsistent'}")

# 4. If inconsistent, implement correction plan (using feature decoupling as example)
if not is_consistent:
    print("\nImplementing correction plan: Feature decoupling (separating truck_allocation from confounding features)")
    
    # Feature decoupling: Use residual method to eliminate confounding between truck_allocation and weather_severity (assuming they are correlated)
    # Step 1: Predict weather_severity using truck_allocation, take residuals as "deconfounded weather feature"
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(X_train[["truck_allocation"]], X_train["weather_severity"])
    X_train["weather_severity_resid"] = X_train["weather_severity"] - lr.predict(X_train[["truck_allocation"]])
    X_val["weather_severity_resid"] = X_val["weather_severity"] - lr.predict(X_val[["truck_allocation"]])
    X_interv["weather_severity_resid"] = X_interv["weather_severity"] - lr.predict(X_interv[["truck_allocation"]])
    
    # Step 2: Retrain model with new features (replacing original weather_severity)
    new_features = [f for f in features if f != "weather_severity"] + ["weather_severity_resid"]
    model_corrected = GradientBoostingClassifier(random_state=42)
    model_corrected.fit(X_train[new_features], y_train)
    
    # Step 3: Re-test causal consistency
    def corrected_causal_test(model, X_interv, new_features, actionable_feature="truck_allocation"):
        X_interv_0 = X_interv.copy()
        X_interv_0[actionable_feature] = 0
        X_interv_1 = X_interv.copy()
        X_interv_1[actionable_feature] = 1
        prob_0 = model.predict_proba(X_interv_0[new_features])[:, 1]
        prob_1 = model.predict_proba(X_interv_1[new_features])[:, 1]
        return {
            "Corrected proportion consistent with business direction": round(np.mean(prob_1 < prob_0), 3),
            "Corrected average probability reduction": round(np.mean(prob_0 - prob_1), 3)
        }
    
    corrected_result = corrected_causal_test(model_corrected, X_interv, new_features)
    print("\nCausal Consistency Results After Correction:")
    for k, v in corrected_result.items():
        print(f"{k}: {v}")

Baseline Model AUC on Validation Set: 0.949

Causal Consistency Test Results:
Proportion consistent with business direction: 0.0
Average reduction in delay probability: -0.0

Causal Consistency Conclusion: Inconsistent

Implementing correction plan: Feature decoupling (separating truck_allocation from confounding features)

Causal Consistency Results After Correction:
Corrected proportion consistent with business direction: 0.0
Corrected average probability reduction: -0.001
